# VQE Warm-Start + Krylov Quantum Diagonalization (KQD)

This notebook: downloads the Fermi-Hubbard Hamiltonian from HamLib, obtains a VQE warm-start state using `AerSimulator`, then runs sample-based Krylov Quantum Diagonalization (KQD) using `PauliEvolutionGate` with `LieTrotter`.

In [ ]:
# Imports and helper functions
import zipfile
import requests
from io import BytesIO
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import warnings
import openfermion as of

from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit.circuit.library import PauliEvolutionGate, TwoLocal
from qiskit.synthesis import LieTrotter
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator
from qiskit.primitives import BaseEstimatorV2
from qiskit_addon_sqd.counts import counts_to_arrays
from qiskit_addon_sqd.qubit import solve_qubit
from collections import Counter
warnings.filterwarnings("ignore")
print("✓ Imports ready")

In [ ]:
import hamlib.hamlib_snippets as hs
hdf5_file = "hamlib/condensedmatter/fermihubbard/FH_D-1.hdf5"
group_key = "/fh-graph-1D-grid-nonpbc-qubitnodes_Lx-5_U-12_enc-jw"

H_of = hs.read_openfermion_hdf5(
    fname_hdf5=hdf5_file,
    key=group_key,
    optype=of.QubitOperator
)

print(f"Loaded Hamiltonian with {len(H_of.terms)} Pauli terms")

In [ ]:
# Convert OpenFermion QubitOperator to Qiskit SparsePauliOp
def of_to_qiskit(op):
    n_qubits = 0
    if len(op.terms) > 0:
        n_qubits = max(q for term in op.terms for q, _ in term) + 1
    labels, coeffs = [], []
    for term, coeff in op.terms.items():
        pauli = ["I"] * n_qubits
        for q, p in term:
            pauli[q] = p
        labels.append("".join(pauli[::-1]))
        coeffs.append(coeff)
    return SparsePauliOp(labels, coeffs)
H_op = of_to_qiskit(H_of)
n_qubits = H_op.num_qubits
# Exact diagonalization for reference (may be large, use if feasible)
H_matrix = np.array(H_op.to_matrix())
eigenvalues, eigenvectors = np.linalg.eigh(H_matrix)
exact_gs_energy = eigenvalues[0]
print(f"System size: {n_qubits} qubits")
print(f"Exact ground state energy (reference): {exact_gs_energy:.8f}")

In [ ]:
# VQE warm-start using Hardware Efficient Ansatz (HEA) and classical optimization
from qiskit.quantum_info import Statevector
from scipy.optimize import minimize

# HEA builder 
def create_hea_ansatz(n_qubits, params, n_layers=1):
    qc = QuantumCircuit(n_qubits)
    param_idx = 0
    for layer in range(n_layers):
        for i in range(n_qubits):
            if param_idx < len(params):
                qc.ry(params[param_idx], i)
                param_idx += 1
            if param_idx < len(params):
                qc.rz(params[param_idx], i)
                param_idx += 1
        for i in range(n_qubits - 1):
            qc.cx(i, i + 1)
    for i in range(n_qubits):
        if param_idx < len(params):
            qc.ry(params[param_idx], i)
            param_idx += 1
        if param_idx < len(params):
            qc.rz(params[param_idx], i)
            param_idx += 1
    return qc

# VQE configuration matching the reference notebook
n_layers = 3
n_params = 2 * n_qubits * (n_layers + 1)

def vqe_cost_function(params):
    qc = create_hea_ansatz(n_qubits, params, n_layers)
    psi = Statevector.from_instruction(qc)
    return np.real(np.vdot(psi.data, H_matrix @ psi.data))

print("Running VQE optimization (classical optimizer, Statevector)...")
best_params = None
best_energy = np.inf
n_random_restarts = 20
np.random.seed(42)
for trial in range(n_random_restarts):
    init_params = np.random.uniform(-np.pi, np.pi, n_params)
    result = minimize(vqe_cost_function, init_params, method='COBYLA', options={'maxiter': 1000, 'rhobeg': 0.5})
    if result.fun < best_energy:
        best_energy = result.fun
        best_params = result.x
    print(f"  Trial {trial+1}/{n_random_restarts}: Energy = {result.fun:.6f}")

vqe_energy = best_energy
vqe_error = abs(best_energy - exact_gs_energy)
# Build VQE-prep circuit and statevector
qc_vqe = create_hea_ansatz(n_qubits, best_params, n_layers)
psi_vqe = Statevector.from_instruction(qc_vqe).data
# Use the VQE circuit as the warm-start state prep
qc_warm = qc_vqe
print(f"VQE best energy: {vqe_energy:.6f}")
print(f"Exact ground state: {exact_gs_energy:.6f}")
print(f"VQE error: {vqe_error:.6f}")

In [ ]:
# Krylov configuration for Hadamard-test-based KQD
krylov_dim = 5
dt = 0.3
num_trotter_steps = 8
num_shots = 50000
# Evolution gate for one time step dt (we will use U^k = exp(-i H k dt))
evol_gate = PauliEvolutionGate(H_op, time=dt, synthesis=LieTrotter(reps=num_trotter_steps))
qr = QuantumRegister(n_qubits)
qc_evol = QuantumCircuit(qr)
qc_evol.append(evol_gate, qargs=qr)
# Decompose for simulator compatibility
qc_evol_decomposed = qc_evol.decompose().decompose()
# Build state circuits |psi_k> = U^k |psi_warm> (no measurements)
state_circuits = []
for rep in range(krylov_dim):
    circ = qc_warm.copy()
    for _ in range(rep):
        circ.compose(other=qc_evol_decomposed, inplace=True)
    state_circuits.append(circ)
print(f"Built {len(state_circuits)} Krylov state circuits (no measurements)")

In [ ]:
# Prepare AerSimulator for Hadamard tests
simulator = AerSimulator()
print("Simulator ready for Hadamard tests (AerSimulator)")

In [ ]:
# Hadamard test builders and estimators 
def build_hadamard_test_real(circuit_i, circuit_j, n_qubits):
    qc = QuantumCircuit(2 * n_qubits + 1, 1)
    ancilla = 2 * n_qubits
    qc.compose(circuit_i, qubits=range(n_qubits), inplace=True)
    qc.compose(circuit_j, qubits=range(n_qubits, 2 * n_qubits), inplace=True)
    qc.h(ancilla)
    for k in range(n_qubits):
        qc.cswap(ancilla, k, n_qubits + k)
    qc.h(ancilla)
    qc.measure(ancilla, 0)
    return qc

def build_hadamard_test_imag(circuit_i, circuit_j, n_qubits):
    qc = QuantumCircuit(2 * n_qubits + 1, 1)
    ancilla = 2 * n_qubits
    qc.compose(circuit_i, qubits=range(n_qubits), inplace=True)
    qc.compose(circuit_j, qubits=range(n_qubits, 2 * n_qubits), inplace=True)
    qc.h(ancilla)
    qc.sdg(ancilla)
    for k in range(n_qubits):
        qc.cswap(ancilla, k, n_qubits + k)
    qc.h(ancilla)
    qc.measure(ancilla, 0)
    return qc

def build_pauli_hadamard_test(circuit_i, circuit_j, pauli_string, n_qubits):
    qc_real = QuantumCircuit(2 * n_qubits + 1, 1)
    ancilla = 2 * n_qubits
    qc_real.compose(circuit_i, qubits=range(n_qubits), inplace=True)
    qc_real.compose(circuit_j, qubits=range(n_qubits, 2 * n_qubits), inplace=True)
    qc_real.h(ancilla)
    for k, p in enumerate(reversed(pauli_string)):
        if p == 'X':
            qc_real.cx(ancilla, n_qubits + k)
        elif p == 'Y':
            qc_real.cy(ancilla, n_qubits + k)
        elif p == 'Z':
            qc_real.cz(ancilla, n_qubits + k)
    for k in range(n_qubits):
        qc_real.cswap(ancilla, k, n_qubits + k)
    qc_real.h(ancilla)
    qc_real.measure(ancilla, 0)
    qc_imag = QuantumCircuit(2 * n_qubits + 1, 1)
    qc_imag.compose(circuit_i, qubits=range(n_qubits), inplace=True)
    qc_imag.compose(circuit_j, qubits=range(n_qubits, 2 * n_qubits), inplace=True)
    qc_imag.h(ancilla)
    qc_imag.sdg(ancilla)
    for k, p in enumerate(reversed(pauli_string)):
        if p == 'X':
            qc_imag.cx(ancilla, n_qubits + k)
        elif p == 'Y':
            qc_imag.cy(ancilla, n_qubits + k)
        elif p == 'Z':
            qc_imag.cz(ancilla, n_qubits + k)
    for k in range(n_qubits):
        qc_imag.cswap(ancilla, k, n_qubits + k)
    qc_imag.h(ancilla)
    qc_imag.measure(ancilla, 0)
    return qc_real, qc_imag

def estimate_overlap_shots(circuit_i, circuit_j, n_qubits, shots=num_shots):
    qc_real = build_hadamard_test_real(circuit_i, circuit_j, n_qubits)
    job_real = simulator.run(qc_real, shots=shots)
    counts_real = job_real.result().get_counts()
    p0_real = counts_real.get('0', 0) / shots
    p1_real = counts_real.get('1', 0) / shots
    real_part = p0_real - p1_real
    qc_imag = build_hadamard_test_imag(circuit_i, circuit_j, n_qubits)
    job_imag = simulator.run(qc_imag, shots=shots)
    counts_imag = job_imag.result().get_counts()
    p0_imag = counts_imag.get('0', 0) / shots
    p1_imag = counts_imag.get('1', 0) / shots
    imag_part = p0_imag - p1_imag
    return real_part + 1j * imag_part

def estimate_pauli_expectation_shots(circuit_i, circuit_j, pauli_string, n_qubits, shots=num_shots):
    qc_real, qc_imag = build_pauli_hadamard_test(circuit_i, circuit_j, pauli_string, n_qubits)
    job_real = simulator.run(qc_real, shots=shots)
    counts_real = job_real.result().get_counts()
    p0_real = counts_real.get('0', 0) / shots
    p1_real = counts_real.get('1', 0) / shots
    real_part = p0_real - p1_real
    job_imag = simulator.run(qc_imag, shots=shots)
    counts_imag = job_imag.result().get_counts()
    p0_imag = counts_imag.get('0', 0) / shots
    p1_imag = counts_imag.get('1', 0) / shots
    imag_part = p0_imag - p1_imag
    return real_part + 1j * imag_part

print(f"Hadamard test functions defined for {n_qubits}-qubit system; shots={num_shots}")

In [ ]:
# Run Hadamard-test-based KQD (estimate S and H_tilde via Hadamard tests)
from scipy.linalg import eigh

def run_kqd_shots(state_circuits, H_op, n_qubits, shots=num_shots):
    K = len(state_circuits)
    # Extract Pauli terms from H_op
    pauli_terms = [(p.to_label(), c) for p, c in zip(H_op.paulis, H_op.coeffs)]
    S = np.zeros((K, K), dtype=complex)
    H_tilde = np.zeros((K, K), dtype=complex)
    for i in range(K):
        for j in range(i, K):
            print(f"Computing S[{i},{j}] and H[{i},{j}]...", end=" ")
            if i == j:
                S[i, j] = 1.0
            else:
                S[i, j] = estimate_overlap_shots(state_circuits[i], state_circuits[j], n_qubits, shots)
                S[j, i] = np.conj(S[i, j])
            h_elem = 0.0
            for pauli_label, coeff in pauli_terms:
                if pauli_label == 'I' * n_qubits:
                    h_elem += coeff * S[i, j]
                else:
                    exp_val = estimate_pauli_expectation_shots(state_circuits[i], state_circuits[j], pauli_label, n_qubits, shots)
                    h_elem += coeff * exp_val
            H_tilde[i, j] = h_elem
            if i != j:
                H_tilde[j, i] = np.conj(H_tilde[i, j])
            print(f"S={S[i,j]:.3f}, H={H_tilde[i,j]:.3f}")
    # Build KQD energies with regularization
    kqd_energies = []
    for k in range(1, K + 1):
        S_k = 0.5 * (S[:k, :k] + S[:k, :k].T.conj())
        H_k = 0.5 * (H_tilde[:k, :k] + H_tilde[:k, :k].T.conj())
        eigvals_S, eigvecs_S = eigh(S_k)
        max_eigval = np.max(eigvals_S)
        threshold = max(1e-2 * max_eigval, 1e-4)
        valid = eigvals_S > threshold
        if np.sum(valid) == 0:
            kqd_energies.append(np.nan)
            continue
        S_inv_sqrt = eigvecs_S[:, valid] @ np.diag(1.0 / np.sqrt(eigvals_S[valid])) @ eigvecs_S[:, valid].T.conj()
        H_transformed = S_inv_sqrt @ H_k @ S_inv_sqrt
        H_transformed = 0.5 * (H_transformed + H_transformed.T.conj())
        eigvals_K, _ = eigh(H_transformed)
        kqd_energies.append(np.real(np.min(eigvals_K)))
    return kqd_energies, S, H_tilde

# Execute KQD (reduce K if needed for tractability)
state_subset = state_circuits[:krylov_dim]
print(f"Running Hadamard-test-based KQD with K={krylov_dim}, shots={num_shots}")
kqd_energies, S_mat, H_mat = run_kqd_shots(state_subset, H_op, n_qubits, shots=num_shots)
print("KQD complete")

In [ ]:
# Results and plots for Hadamard-test-based KQD
print("FINAL RESULTS: Hadamard-test-based KQD")
print(f"Exact GS Energy: {exact_gs_energy:.8f}")
for i, e in enumerate(kqd_energies):
    print(f"K={i+1}: E={e:.8f}, error={abs(e-exact_gs_energy):.6f}")
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(range(1, len(kqd_energies)+1), kqd_energies, marker='o', label='KQD Estimate')
ax.axhline(y=exact_gs_energy, color='r', linestyle='--', label='Exact GS')
ax.set_xlabel('Krylov dimension (K)')
ax.set_ylabel('Energy')
ax.legend()
ax.grid(True)
plt.show()

*This code was part of the work done as part of the Qiskit Advocate Mentorship Programme (QAMP) 2025 project No.: 31.*\
*Mentors: Dr. Soham Pal, Dr. Shiplu Sarker,*\
*Mentees: Abdullah Afzal, Michael Papadopoulos, Gayathree M. Vinod.*\
*This notebook was prepared by Abdullah Afzal and verified by Michael Papadopoulos.*